## Medical Expense Prediction

Medical expenses is one of the major expenses in an individual's life. We know that one's life style and various physiological charactreristics causes diseases or ailments one can have and these ailments influence medical expanses. According to various studies, major factors that contribute to higher expenses in personal medical care include smoking, aging, high BMI. In this study, we aim to find a correlation between personal medical expenses and different factors, and compare them. Then we use the prominent attributes as predictors to predict medical expenses by creating linear regression models.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Set path and read data file
insurance_data = pd.read_csv("medical_insurance.csv", header=0)
insurance_data.drop('Unnamed: 0', axis=1, inplace=True)

Following are the details of each feature/attribute of the given dataset.

**age:** age of primary beneficiary

**sex:** gender - female, male

**bmi:** Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

**children:** Number of children / Number of dependents

**smoker:** Smoking / non-smoker

**region:** the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

**charges:** Individual medical costs billed by health insurance

In [3]:
# Encode categorical columns
categorical_columns = ['sex','children', 'smoker', 'region']
insurance_data_encode = pd.get_dummies(data=insurance_data, columns=categorical_columns, prefix='OHE', prefix_sep='_', drop_first=True)
print('Columns in original data frame:\n',insurance_data.columns.values)
print('\nNumber of rows and columns in the dataset:',insurance_data.shape)
print('\nColumns in data frame after encoding dummy variable:\n',insurance_data_encode.columns.values)
print('\nNumber of rows and columns in the dataset:',insurance_data_encode.shape)

Columns in original data frame:
 ['age' 'sex' 'bmi' 'children' 'smoker' 'region' 'charges']

Number of rows and columns in the dataset: (1338, 7)

Columns in data frame after encoding dummy variable:
 ['age' 'bmi' 'charges' 'OHE_male' 'OHE_1' 'OHE_2' 'OHE_3' 'OHE_4' 'OHE_5'
 'OHE_yes' 'OHE_northwest' 'OHE_southeast' 'OHE_southwest']

Number of rows and columns in the dataset: (1338, 13)


In [4]:
# Create dataframe to save the results of each model
results_df = pd.DataFrame([], columns=['model','train_error', 'test_error', 'Train_r2', 'Test_r2'])

In [5]:
# Split dataset into train & test sets in ratio 70:30
X = insurance_data_encode.drop(columns=['charges']) # Independent features
y = insurance_data_encode['charges'] # Dependent features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)  

##### Linear Regression with all variables

In [6]:
# Fit a linear regression model with all features and without scaling
lr = LinearRegression()
lr.fit(X_train, y_train)

test_pred = lr.predict(X_test)
train_pred = lr.predict(X_train)

train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)

In [7]:
results_df.loc[len(results_df.index)] = ['LR_All Vars', train_rmse, test_rmse, train_r2, test_r2]
results_df

,model,train_error,test_error,Train_r2,Test_r2
0,LR_All Vars,6087.120951,5928.918519,0.75787,0.730528


##### Linear Regression with scaled data

In [8]:
# Scale Data
scaler = MinMaxScaler()
X_train[['age', 'bmi']] = scaler.fit_transform(X_train[['age', 'bmi']])
X_test[['age', 'bmi']] = scaler.transform(X_test[['age', 'bmi']])

In [9]:
lr1 = LinearRegression()
lr1.fit(X_train, y_train)

test_pred = lr1.predict(X_test)
train_pred = lr1.predict(X_train)

train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)

In [10]:
results_df.loc[len(results_df.index)] = ['LR_Scaled All Vars', train_rmse, test_rmse, train_r2, test_r2]
results_df

,model,train_error,test_error,Train_r2,Test_r2
0,LR_All Vars,6087.120951,5928.918519,0.75787,0.730528
1,LR_Scaled All Vars,6087.120951,5928.918519,0.75787,0.730528


**Observation:** No improvement in training/test error due to scaling

##### With only correlated features

In [11]:
# Split dataset into train & test sets
X = insurance_data_encode[['age', 'bmi', 'OHE_yes']] # Independent features
y = insurance_data_encode['charges'] # Dependent features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)  

In [12]:
lr2 = LinearRegression()
lr2.fit(X_train, y_train)

test_pred = lr2.predict(X_test)
train_pred = lr2.predict(X_train)

train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))

In [13]:
results_df.loc[len(results_df.index)] = ['LR_Correlated', train_rmse, test_rmse, lr2.score(X_train, y_train), lr2.score(X_test, y_test) ]
results_df

,model,train_error,test_error,Train_r2,Test_r2
0,LR_All Vars,6087.120951,5928.918519,0.757870,0.730528
1,LR_Scaled All Vars,6087.120951,5928.918519,0.757870,0.730528
2,LR_Correlated,6157.675120,5920.647746,0.752225,0.731280


##### Fit a polynomial model

We can now increase the model complexity by including polynomial features of various degrees. In order to find the best polynomial degree that would best the dataset, we run the regression model on different polynomial degree features from 1 to 10.

In [14]:
def fit_poly(train, y_train, test, y_test, degrees):

    features = PolynomialFeatures(degree=degrees, include_bias=False)
    train_trans = features.fit_transform(train)
    
    model = LinearRegression()
    model.fit(train_trans, y_train)
    
    train_predictions = model.predict(train_trans)
    training_error = np.sqrt(mean_squared_error(y_train, train_predictions))
    
    test_trans = features.fit_transform(test)
    test_predictions = model.predict(test_trans)
    testing_error = np.sqrt(mean_squared_error(y_test, test_predictions))
    
    return training_error, testing_error, r2_score(y_train, train_predictions), r2_score(y_test, test_predictions)

In [15]:
# Split dataset into train & test sets
X = insurance_data_encode.drop(columns=['charges']) # Independent features
y = insurance_data_encode['charges'] # Dependent features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)  

In [16]:
degrees = [int(x) for x in np.linspace(1, 10, 10)]
poly_df = pd.DataFrame(0, columns=['train_error', 'test_error'], index=degrees)
for degree in degrees:
    degree_results = fit_poly(X_train, y_train, X_test, y_test, degrees = degree)
    poly_df.loc[degree, 'train_error'] = degree_results[0]
    poly_df.loc[degree, 'test_error'] = degree_results[1]
poly_df

,train_error,test_error
1,6087.120951,5.928919e+03
2,4601.114368,4.913276e+03
3,4245.508867,7.043651e+03
4,3753.152871,7.193522e+03
5,2915.949843,9.423187e+04
6,2152.778581,9.225749e+06
7,1875.442558,1.010082e+07
8,1349.549926,1.404983e+07
9,1201.230103,7.451754e+06
10,1002.873261,5.691233e+06


**Observation:** Training error keeps decreasing on as degree of polynomial increases. But test error drops till degree 2 and then increases. Thus as degree of polynomials increases, there is overfitting on training data. Hence the model performs well on training data but performs poorly on unseen test data. Hence polynomial of degree 2 is suitable. 

##### Linear Regression - All vars, Polynomial Degree 2

In [17]:
degree_results = fit_poly(X_train, y_train, X_test, y_test, degrees = 2)
results_df.loc[len(results_df.index)] = ['LR_Poly2', degree_results[0], degree_results[1], degree_results[2], degree_results[3]]
results_df

,model,train_error,test_error,Train_r2,Test_r2
0,LR_All Vars,6087.120951,5928.918519,0.757870,0.730528
1,LR_Scaled All Vars,6087.120951,5928.918519,0.757870,0.730528
2,LR_Correlated,6157.675120,5920.647746,0.752225,0.731280
3,LR_Poly2,4601.114368,4913.275630,0.861659,0.814944


##### Linear Regression with correlated features - Polynomial features

In [18]:
# Split dataset into train & test sets
X = insurance_data_encode[['age', 'bmi', 'OHE_yes']] # Independent features
y = insurance_data_encode['charges'] # Dependent features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)  

degrees = [int(x) for x in np.linspace(1, 10, 10)]
poly_df = pd.DataFrame(0, columns=['train_error', 'test_error'], index=degrees)
for degree in degrees:
    degree_results = fit_poly(X_train, y_train, X_test, y_test, degrees = degree)
    poly_df.loc[degree, 'train_error'] = degree_results[0]
    poly_df.loc[degree, 'test_error'] = degree_results[1]
poly_df

,train_error,test_error
1,6157.675120,5920.647746
2,4834.039923,4950.455269
3,4782.974028,4973.950158
4,4716.021754,4793.604530
5,4630.413574,4717.864625
6,4572.699113,4672.902956
7,4511.568971,5161.615189
8,4510.086175,5738.127398
9,4478.708179,5089.207041
10,4498.565963,5341.405958


**Observation:** With only correlated features, the test error is minimum for polynomial degree 6

##### Linear Regression - Correlated vars, Polynomial Degree 6

In [19]:
degree_results = fit_poly(X_train, y_train, X_test, y_test, degrees = 6)
results_df.loc[len(results_df.index)] = ['LR_Poly6', degree_results[0], degree_results[1], degree_results[2], degree_results[3]]
results_df

,model,train_error,test_error,Train_r2,Test_r2
0,LR_All Vars,6087.120951,5928.918519,0.757870,0.730528
1,LR_Scaled All Vars,6087.120951,5928.918519,0.757870,0.730528
2,LR_Correlated,6157.675120,5920.647746,0.752225,0.731280
3,LR_Poly2,4601.114368,4913.275630,0.861659,0.814944
4,LR_Poly6,4572.699113,4672.902956,0.863363,0.832608


**Conclusion:** Fitting a polynomial of degree 6 using correlated features gives the best model performance for this dataset